# Aims

The purpose of this notebook is to reproduce the paper and check how it applies to the present data.

## Imports

In [1]:
import pandas_datareader.data as web
import datetime as dt

# Pick the companies

In [2]:
companies_to_tickers = {"Nike": "NKE", 
                       "Goldman Sachs": "GS", 
                       "Johnson and Johnson": "JNJ", 
                       "Pfizer": "PFE",
                       "JP Morgan Chase and Co": "JPM"}

In [3]:
tickers = list(companies_to_tickers.values())
tickers

['NKE', 'GS', 'JNJ', 'PFE', 'JPM']

# Data

## Load

In [4]:
start_train = dt.datetime(2009, 4, 6)
end_train = dt.datetime(2017, 4, 3)
start_test = dt.datetime(2017, 4, 4)
end_test = dt.datetime(2019, 4, 5)

In [5]:
data_train = {ticker: web.DataReader(ticker, "yahoo", start_train, end_train)
             for ticker in tickers}
data_test = {ticker: web.DataReader(ticker, "yahoo", start_test, end_test)
             for ticker in tickers}

In [6]:
print(tickers[0])
data_train[tickers[0]].head()

NKE


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2009-04-06,12.8750,12.6150,12.8200,12.8175,11080000.0,10.948364
2009-04-07,12.5950,12.3775,12.5925,12.4650,11580000.0,10.647271
2009-04-08,12.6550,12.3625,12.4900,12.5600,13103200.0,10.728414
2009-04-09,13.3750,12.6450,12.8125,13.3175,18044400.0,11.375454
2009-04-13,13.3675,12.8975,13.1825,13.2800,13202000.0,11.343422


## Add new variables

In [7]:
def add_new_vars(data):
    for ticker in tickers:
        data[ticker]["H-L"] = data[ticker]["High"] - data[ticker]["Low"]
        data[ticker]["O-C"] = data[ticker]["Open"] - data[ticker]["Close"]
        # Stock price’s seven days’ moving average (7 DAYS MA)
        # there is not explicitly said of what
        # so let's assume that it would be of the close price
        data[ticker]["MA-7d"] = data[ticker]["Close"].rolling(window=7).mean()
        data[ticker]["MA-14d"] = data[ticker]["Close"].rolling(window=14).mean()
        data[ticker]["MA-21d"] = data[ticker]["Close"].rolling(window=21).mean()
        data[ticker]["std-7d"] = data[ticker]["Close"].rolling(window=7).std()

In [8]:
add_new_vars(data_train)
add_new_vars(data_test)

# Model

## Artificial Neural Network

Since the detailed descritption is not provided about the training I'll infer the architecture from the model image and adjust the training time (epochs) and other hyperparams according to my preference.

In [9]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input

In [10]:
model = Sequential()
model.add(Input(shape=(7,)))
model.add(Dense(3))
model.add(Dense(1))

2021-12-25 02:59:44.002710: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 24        
                                                                 
 dense_1 (Dense)             (None, 1)                 4         
                                                                 
Total params: 28
Trainable params: 28
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer="sgd", loss='mse')